In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#需要的套件
import pandas as pd
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import math
import matplotlib
import datetime
## scikit learn part
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE
from imblearn.combine import SMOTEENN
from sklearn import preprocessing
from scipy.stats.mstats import winsorize
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import ClusterCentroids
## scikit learn matrics
from sklearn import metrics
import xgboost as xgb
import lightgbm as lgb
## ml part
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from IPython.display import display
import pandas as pd


In [ ]:
X = pd.read_csv('/content/drive/My Drive/FINTECH Final/train_ver_final.csv')
Xtest = pd.read_csv('/content/drive/My Drive/FINTECH Final/test_ver_final.csv')
df = pd.read_csv('/content/drive/My Drive/FINTECH Final/dataset_2nd/private_1_processed.csv')
test_answer = pd.read_csv('/content/drive/My Drive/FINTECH Final/dataset_2nd/public.csv') # predict answer

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate



In [ ]:
y = pd.read_csv('/content/drive/My Drive/FINTECH Final/ylabel.csv')
newy = pd.read_csv('/content/drive/My Drive/FINTECH Final/dataset_2nd/public.csv')
print(y.shape)
print(newy.shape)
newy = newy['label'].to_frame()
newy.rename(columns={0: 'label'})
# print(y.head(5))
print(newy.head(5))

(8688526, 1)
(609040, 26)
   label
0      0
1      0
2      0
3      0
4      0
(8688526, 1)


In [ ]:
y = pd.concat([y, newy])
print(y.shape)
print(y.head(5))

(9297566, 1)
   label
0      0
1      0
2      0
3      0
4      0


In [ ]:
print(X.shape)
print(Xtest.shape)
print(df.shape)

(9297566, 48)
(754139, 48)
(754139, 25)


In [ ]:
y = df.pop('label')
X = df.drop(['txkey'], axis=1)
id = Xtest['txkey']
Xtest = Xtest.drop(['txkey'], axis=1)

In [ ]:
parameters = {'max_depth':[5],
        #'max_delta_step': [0, 0.6, 1],
        #'n_estimators': [1000, 1500, 2000]
        # 'colsample_bytree': [0.8, 0.9],
              #'reg_alpha': [0,  0.5,  1],
              #'reg_lambda': [0.2,  0.6,  1],
              #'scale_pos_weight': [0.2, 0.6, 1]

}

In [ ]:
from sklearn.metrics import make_scorer, f1_score
xlf = xgb.XGBClassifier(
      device="cuda",
			learning_rate=0.05,
			max_depth=10,
			n_estimators=2000,
			verbosity=0,
			objective='binary:logistic',
			nthread=-1,
			gamma=0.1,
			min_child_weight=0,
			subsample=0.9,
			colsample_bytree=0.85,
			colsample_bylevel=1,
			eval_metric=f1_score,
			tree_method="hist",
			seed=523,
      )

In [ ]:
from sklearn.metrics import make_scorer, f1_score

f1_scorer = make_scorer(f1_score)
gsearch = GridSearchCV(xlf, param_grid=parameters, scoring=f1_scorer, cv=5, verbose=3)
gsearch.fit(X, y)

print("Best score: %0.5f" % gsearch.best_score_)
print("Best parameters set:")
best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
# !pip install joblib
from sklearn.metrics import f1_score

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# xlf = xgb.XGBClassifier(
#       device="cuda",
# 			learning_rate=0.001,
#       max_depth=10,
# 			n_estimators=2000,
# 			verbosity=3,
# 			objective='binary:logistic',
# 			nthread=-1,
# 			gamma=0,
# 			min_child_weight=0,
# 			subsample=0.85,
# 			colsample_bytree=0.7,
# 			colsample_bylevel=1,
# 			eval_metric=f1_score,
# 			seed=523,
# )

xlf.fit(X_train, y_train)

# Make predictions
y_pred = xlf.predict(X_val)

# Evaluate the model
accuracy = f1_score(y_val, y_pred)
print(f'f1 score on validation: {accuracy:.5f}')


f1 score on validation: 0.84888


In [ ]:
#test = pd.read_csv('/content/drive/My Drive/FINTECH Final/test.csv')
y_pred = xlf.predict(Xtest)
id = df['txkey']
df = {"txkey":id, "label": y_pred}
df = pd.DataFrame(df)
# merged_df = pd.merge(df, test_answer[["txkey", "label"]], on='txkey', how='left')



                                               txkey  label
0  a069fe3514177af67c02eb9f0f49c3f44a59fbc3299e74...      0
1  e7a54bf48f9483847b4632512c27a5d0793c2927a12af1...      0
2  90cbca23c88d66ccd7de27ee2f658a8ba1dbcbd9908889...      0
3  1ef5adcad1d3ad7cd6cbb4bc691fe1805900d44f791a56...      0
4  10f0d86f50e37ba7de0ac518d09c0331ce3a3f092e1649...      0


In [ ]:
print(df.shape)
df.to_csv("/content/drive/My Drive/FINTECH Final/XGboost_private.csv", index=False)
print('Export Succeeded')

(754139, 2)
Export Succeeded


In [ ]:
merged_df = merged_df.rename(columns={'label_x': 'predict', 'label_y': 'real_label'})
merged_df.head(5)

,txkey,predict,real_label
0,a2c1209018e4e52e04f6fabb48f05f1b8bc09dc838ff6c...,0,0
1,16c4880500059e01553789be11bbb50753b7acaae7b95b...,0,0
2,623c56be3bee87724e3d119c271d9ed098eeda84233183...,0,0
3,250da12187059cf6e3a3066656a2919d08ceb8207efd55...,0,0
4,4b268e0da036f44cbbb056ddfac6a28ea336d9cf299843...,0,0


In [ ]:
print("f1 score on public =")
f1_score(merged_df["predict"], merged_df["real_label"])

f1 score on public =


0.6412257877999422

In [ ]:
# test = pd.read_csv('/content/drive/My Drive/FINTECH Final/test.csv')
y_pred = xlf.predict(Xtest)
id = test['txkey']
df = {"txkey":id, "label": y_pred}
df = pd.DataFrame(df)
df.to_csv("/content/drive/My Drive/FINTECH Final/XGboost(2).csv", index=False)
print('Export Succeeded')

Export Succeeded


In [ ]:
import joblib
model_filename = '/content/drive/My Drive/FINTECH Final/XGboost.joblib'
joblib.dump(xlf, model_filename)
print(f'Model saved to {model_filename}')

Model saved to /content/drive/My Drive/FINTECH Final/XGboost.joblib


In [ ]:
Xtest = pd.read_csv('/content/drive/My Drive/FINTECH Final/test_ver3.csv')
test = pd.read_csv('/content/drive/My Drive/FINTECH Final/test.csv')
id = test['txkey']
y_pred = xlf.predict(Xtest)
dfpublic = {"txkey":id, "label": y_pred}
dfpublic = pd.DataFrame(dfpublic)


In [ ]:
print(df.shape)
print(dfpublic.shape)

(754139, 2)
(600182, 2)


In [ ]:
finaldf = pd.concat([dfpublic, df])
finaldf.shape

(1354321, 2)

In [ ]:
finaldf.to_csv("/content/drive/My Drive/FINTECH Final/XGboost_final.csv", index=False)